In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

# Read the CSV and Perform Basic Data Cleaning

In [4]:
# https://help.lendingclub.com/hc/en-us/articles/215488038-What-do-the-different-Note-statuses-mean-

columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [36]:
# Load the data
df = pd.read_csv('LoanStats_2019Q1.csv', skiprows=1)[:-2]
df = df.loc[:,columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna(axis='rows', how='all')

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period','Fully Paid','Charged Off'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,35000.0,0.1308,797.80,MORTGAGE,125890.0,Verified,Mar-2019,high_risk,n,30.48,...,100.0,80.0,0.0,0.0,399376.0,131350.0,32700.0,130876.0,N,N
1,20000.0,0.2250,769.00,RENT,75000.0,Not Verified,Mar-2019,low_risk,n,24.37,...,100.0,100.0,0.0,0.0,77797.0,55014.0,47000.0,30797.0,N,N
2,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
3,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
4,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N


In [6]:
X = df.copy()
X = X.drop(axis = 1, columns = 'loan_status')
X = pd.get_dummies(X, columns=["home_ownership", "verification_status", "issue_d", "pymnt_plan", "initial_list_status","next_pymnt_d","application_type","hardship_flag","debt_settlement_flag"])
X.head()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,issue_d_Mar-2019,pymnt_plan_n,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N
0,35000.0,0.1308,797.80,125890.0,30.48,0.0,0.0,14.0,0.0,32363.0,...,1,1,0,1,0,0,1,0,1,1
1,20000.0,0.2250,769.00,75000.0,24.37,0.0,0.0,8.0,0.0,44934.0,...,1,1,0,1,0,1,1,0,1,1
2,10500.0,0.1719,375.35,66000.0,27.24,0.0,0.0,8.0,0.0,1609.0,...,1,1,0,1,0,1,1,0,1,1
3,25000.0,0.2000,929.09,105000.0,20.23,0.0,0.0,17.0,1.0,18368.0,...,1,1,0,1,0,1,1,0,1,1
4,20000.0,0.2000,529.88,56000.0,24.26,0.0,0.0,8.0,0.0,13247.0,...,1,1,0,1,0,1,1,0,1,1


In [7]:
X = X.fillna(0)

In [8]:
# encode loan status column so you can use it for target
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['loan_status'] = le.fit_transform(df['loan_status']) 
df["loan_status"].unique()

array([0, 1], dtype=int64)

In [9]:
# Define the target set.
y = df["loan_status"].ravel()
y[:5]

array([0, 1, 1, 1, 1])

# Split the Data into Training and Testing

In [10]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,issue_d_Mar-2019,pymnt_plan_n,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N
count,96840.000000,96840.000000,96840.000000,9.684000e+04,96840.000000,96840.000000,96840.000000,96840.000000,96840.000000,96840.000000,...,96840.000000,96840.0,96840.000000,96840.000000,96840.000000,96840.000000,96840.000000,96840.000000,96840.0,96840.0
mean,16642.454822,0.126793,480.642858,8.516747e+04,20.270789,0.226301,0.443680,11.707239,0.117627,17836.567803,...,0.178346,1.0,0.124153,0.875847,0.374349,0.602953,0.858870,0.141130,1.0,1.0
std,10365.336053,0.048470,291.474838,1.133767e+05,19.885875,0.740297,0.737118,5.978739,0.327508,22679.674588,...,0.382805,0.0,0.329758,0.329758,0.483957,0.489288,0.348157,0.348157,0.0,0.0
min,1000.000000,0.060000,30.640000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,1.0
25%,9000.000000,0.081900,264.500000,4.900000e+04,12.080000,0.000000,0.000000,7.000000,0.000000,6156.500000,...,0.000000,1.0,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.0,1.0
50%,15000.000000,0.118000,399.590000,7.000000e+04,18.330000,0.000000,0.000000,11.000000,0.000000,12047.000000,...,0.000000,1.0,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.0,1.0
75%,24000.000000,0.155700,648.100000,1.000000e+05,25.640000,0.000000,1.000000,15.000000,0.000000,21892.250000,...,0.000000,1.0,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.0,1.0
max,40000.000000,0.308900,1676.230000,9.000000e+06,999.000000,18.000000,5.000000,72.000000,4.000000,613796.000000,...,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.0


In [11]:
# Check the balance of our target values
print(len(y))

96840


In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

# Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble AdaBoost classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Print the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier onely, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [13]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.datasets import make_classification
clf = BalancedRandomForestClassifier(random_state=1)

In [17]:
# Fitting the model
clf_model = clf.fit(X_train, y_train)

In [18]:
# Making predictions using the testing data.
predictions = clf.predict(X_test)

In [22]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
cm = confusion_matrix(y_test, predictions)
cm

array([[  590,    73],
       [  156, 23391]], dtype=int64)

In [24]:
# Calculated the balanced accuracy score
acc_score = accuracy_score(y_test, predictions)
acc_score

0.9905410987195373

In [25]:
# Print the imbalanced classification report
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.79      0.89      0.84       663
           1       1.00      0.99      1.00     23547

    accuracy                           0.99     24210
   macro avg       0.89      0.94      0.92     24210
weighted avg       0.99      0.99      0.99     24210



In [28]:
# List the features sorted in descending order by feature importance
importances = clf_model.feature_importances_
sorted(zip(clf_model.feature_importances_, X.columns), reverse=True)

[(0.1525806153034264, 'out_prncp'),
 (0.13151452136198352, 'out_prncp_inv'),
 (0.12220904146359449, 'total_pymnt_inv'),
 (0.11597414312772418, 'total_pymnt'),
 (0.09644778494959173, 'total_rec_prncp'),
 (0.09600848416091479, 'last_pymnt_amnt'),
 (0.056196013612352456, 'total_rec_int'),
 (0.02487257888528643, 'next_pymnt_d_May-2019'),
 (0.012822182263067914, 'next_pymnt_d_Apr-2019'),
 (0.008327153272193707, 'installment'),
 (0.007717183374432061, 'int_rate'),
 (0.007665512640824376, 'loan_amnt'),
 (0.007272597078262417, 'bc_util'),
 (0.006838953116371235, 'dti'),
 (0.005601926641537774, 'total_rec_late_fee'),
 (0.005015368715676554, 'mo_sin_old_rev_tl_op'),
 (0.0048343427516387115, 'annual_inc'),
 (0.0046886813206595, 'revol_bal'),
 (0.004621393633882997, 'tot_cur_bal'),
 (0.004458678708936727, 'max_bal_bc'),
 (0.004092239450887472, 'all_util'),
 (0.0040906038944261616, 'avg_cur_bal'),
 (0.004010118820919546, 'total_bal_ex_mort'),
 (0.0038511628709983225, 'bc_open_to_buy'),
 (0.00378762

### Easy Ensemble AdaBoost Classifier

In [31]:
# Train the EasyEnsembleClassifier
from imblearn.ensemble import EasyEnsembleClassifier 
eec = EasyEnsembleClassifier(random_state=42)

eec.fit(X_train, y_train)

EasyEnsembleClassifier(random_state=42)

In [32]:
# Calculated the balanced accuracy score
predictions = eec.predict(X_test)

In [33]:
# Display the confusion matrix
cm = confusion_matrix(y_test, predictions)
cm

array([[  608,    55],
       [  170, 23377]], dtype=int64)

In [34]:
# Calculated the balanced accuracy score
acc_score = accuracy_score(y_test, predictions)
acc_score

0.9907063197026023

In [ ]:
# Print the imbalanced classification report


In [35]:
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.78      0.92      0.84       663
           1       1.00      0.99      1.00     23547

    accuracy                           0.99     24210
   macro avg       0.89      0.95      0.92     24210
weighted avg       0.99      0.99      0.99     24210

